# TESTTESTTESTTESTTESTTESTTESTTESTTESTTEST

In [ ]:
%reload_ext autoreload
%autoreload 2
from hw5_funct import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from numba import jit, autojit
import multiprocessing
from itertools import repeat

X_data, Y_data, activities = get_data('./data/')

In [ ]:
@autojit
def get_accuracy_basic(window, K, X_data, Y_data):
    rfscores = []
    knnscores = []
    for i in range(10):
        X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, stratify=Y_data)
        vq_dict = get_vq_dict(X_train, window, K)
        X_train_VQ = vec_quantize_signals(X_train, vq_dict)
        X_test_VQ = vec_quantize_signals(X_test, vq_dict)
        rf = RandomForestClassifier(n_estimators=100, max_depth=50)
        rf.fit(X_train_VQ, Y_train)
        rfscore = rf.score(X_test_VQ,Y_test)
        rfscores.append(rfscore)
        knn = KNeighborsClassifier(n_neighbors=1)
        knn.fit(X_train_VQ, Y_train)
        knnscore = knn.score(X_test_VQ,Y_test)
        knnscores.append(knnscore)
    return np.mean(rfscores), np.mean(knnscores)

In [ ]:
%%time
windows = [3*32]
kvs = [25, 50, 100, 200, 400]
for window in windows:
    for K in kvs:
        rfscore, knnscore = get_accuracy_basic(window, K, X_data, Y_data)
        print('%d, %d, %.5f, %.5f' % (window, K, rfscore, knnscore))